<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9A%D0%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Конечные автоматы Мили и Мура
Вариант 5

In [2]:
import numpy as np
import pandas as pd
import random
import copy
from prettytable import PrettyTable
import itertools as it

In [3]:
def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

def implic(a,b): # импликация ->
  rez = 1
  if(a == 1 and b == 0): rez = 0
  return rez

def pier(a,b): # стрелка Пирса ^
  rez = 0
  if(a == 0 and b == 0): rez = 1
  return rez

def schaeffer(a,b): # штрих Шеффера |
  rez = 1
  if(a == 1 and b == 1): rez = 0
  return rez

#Очерёдность выполнения логических операций:
#1-Инверсия;
#2-Конъюнкция и штрих Шеффера;
#3-Дизъюнкция и стрелка Пирса;
#4-Импликация и сложение по модулю 2;
#5-Эквивалентность

##Построение таблицы истинности:

In [4]:
#таблица истинности
def truthTable(p_inp, p_st, x, v):
  inner_table = [[(str(x1(p_st[i], p_inp[j])) + str(x2(p_st[i], p_inp[j]))) for j in range(len(p_inp))] for i in range(len(p_st))]

  print('Таблица истинности')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "v \ x")
  table.field_names = columns

  for i in range(len(p_inp)):
    table.add_row([p_inp[i], inner_table[i][0], inner_table[i][1],
                             inner_table[i][2], inner_table[i][3]])

  columns.clear()
  print(table)
  return inner_table

#----------------------------------------------------------------------------------------------------

#таблица выходов для автомата Мура (выход зависит только от состояния)
def outputTable_Mura(p_inp, p_st, x, v):
  inner_table = [(str(y1(p_st[j], p_inp[j])) + str(y2(p_st[j], p_inp[j]))
                  + str(y3(p_st[j], p_inp[j])) + str(y4(p_st[j], p_inp[j]))) for j in range(len(p_inp))]

  print('Таблица выходов')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "y \ x")
  table.field_names = columns

  table.add_row(['y', inner_table[0], inner_table[1], inner_table[2], inner_table[3]])

  columns.clear()
  print(table)
  return inner_table

#таблица выходов для автомата Мили (выход зависит и от состояния, и от входа)
def outputTable_Mili(p_inp, p_st, x, v):
  inner_table = [[(str(y1(p_st[i], p_inp[j])) + str(y2(p_st[i], p_inp[j]))
                  + str(y3(p_st[i], p_inp[j])) + str(y4(p_st[i], p_inp[j]))) for j in range(len(p_inp))] for i in range(len(p_st))]

  print('Таблица выходов')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "v \ x")
  table.field_names = columns

  for i in range(len(p_inp)):
    table.add_row([p_inp[i], inner_table[i][0], inner_table[i][1],
                             inner_table[i][2], inner_table[i][3]])

  columns.clear()
  print(table)
  return inner_table

#----------------------------------------------------------------------------------------------------

#матрица соединений
def creatly_connection_table(trans, out):
  table = pd.DataFrame('0', index=trans.index, columns=trans.columns)
  np_out = np.matrix(out.copy())

  #if(len(out) != 1):
  for i in range(table.shape[0]):
    table['input'][i] = table.columns[i]


  for i in table.columns[0:table.columns.shape[0]-1]:
    for j in range(table.shape[0]):
      cell = trans.index[trans[i] == table[table.columns[table.shape[1]-1]][j]]
      if len(cell) != 0:
        table[i][j] = "";
        for k in range(len(cell)):

          if(len(np_out) > 1):
            table[i][j] += out[out.columns[out.shape[1]-1]][cell[k]] + "/" + out[i][cell[k]]
          else:
            table[i][j] += trans[trans.columns[trans.shape[1]-1]][cell[k]] + "/" + out[i]

          if k != len(cell) - 1:
            table[i][j] += " V "

  table = table.drop('input', axis=1)
  return table

###Задача 1. Мура

In [5]:
#x0 = ['0', '1']

#2 входа - v
v = ["0", "1"]
#2 состояния - x1/x2
x = ['0', '1']
#4 выхода - y1/y2/y3/y4

pos_inputs = []
for i in range(len(v)):
  for j in range(len(v)):
    pos_inputs.append(str(v[i]) + str(v[j]))
print('Возможные входы:', pos_inputs)

pos_states = []
for i in range(len(x)):
  for j in range(len(x)):
    pos_states.append(str(x[i]) + str(x[j]))
print('Возможные состояния:', pos_states)

Возможные входы: ['00', '01', '10', '11']
Возможные состояния: ['00', '01', '10', '11']


In [6]:
#Выходы и состояния для автомата Мура заданы функциями с логическими операторами

def x1(x, v):
  #v1 + (x2 | x1 + (v2 v (v2 + (x2 ^ x2))))

  rezult = int(v[0]) ^ (schaeffer(int(x[1]), int(x[0])) ^ (int(v[1]) or (int(v[1]) ^ pier(int(x[1]), int(x[1])))))
  return rezult

def x2(x, v):
  #v1 -> (v1 | (x1' | v1 + (x1 ^ v2)))

  rezult = implic(int(v[0]), schaeffer(int(v[0]), schaeffer(not int(x[0]), int(v[0])) ^ pier(int(x[0]), int(v[1]))))
  return rezult

def y1(x, v):
  #(x1 v x2) + (x2 | x1) ~ x1 ~ (x2' -> x1)

  rezult = equiv(equiv(int(x[0]) or int(x[1]) ^ schaeffer(int(x[1]), int(x[0])), int(x[0])), implic(not int(x[1]), int(x[0])))
  return rezult

def y2(x, v):
  #x1 ~ x2 ^ (x1 + x2 | (x2 ~ x2))

  rezult = equiv(int(x[0]), pier(int(x[1]), int(x[0]) ^ schaeffer(int(x[1]), equiv(int(x[1]), int(x[1])))))
  return rezult

def y3(x, v):
  #x2 & x2 | x2 + x2 ^ x2

  rezult = schaeffer(int(x[1]) and int(x[1]), int(x[1])) ^ pier(int(x[1]), int(x[1]))
  return rezult

def y4(x, v):
  #x1 v x1 ~ x2 & (x2 ^ x1 ^ x2)

  rezult = equiv(int(x[0]) or int(x[0]), int(x[1]) and pier(pier(int(x[1]), int(x[0])), int(x[1])))
  return rezult

In [7]:
print("Автомат Мура:\n")
truth_table = truthTable(pos_inputs, pos_states, x, v)
print("\n")
output_table = outputTable_Mura(pos_inputs, pos_states, x, v)

Автомат Мура:

Таблица истинности
+-------+----+----+----+----+
| v \ x | 00 | 01 | 10 | 11 |
+-------+----+----+----+----+
|   00  | 01 | 01 | 10 | 11 |
|   01  | 11 | 01 | 00 | 11 |
|   10  | 01 | 01 | 10 | 10 |
|   11  | 01 | 11 | 10 | 00 |
+-------+----+----+----+----+


Таблица выходов
+-------+------+------+------+------+
| y \ x |  00  |  01  |  10  |  11  |
+-------+------+------+------+------+
|   y   | 1101 | 1101 | 1100 | 1000 |
+-------+------+------+------+------+


In [8]:
truth_copy = pd.DataFrame(truth_table.copy(), columns = pos_states, index = pos_inputs)
truth_copy['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=truth_copy.index)

out_copy = pd.DataFrame(np.matrix(output_table).copy(), columns = pos_states)

creatly_connection_table(truth_copy, out_copy)

,00,01,10,11
00,0,0,[01/1100],[11/1000]
01,[00/1101 V 10/1101 V 11/1101],[00/1101 V 01/1101 V 10/1101],0,0
10,0,0,[00/1100 V 10/1100 V 11/1100],[10/1000]
11,[01/1101],[11/1101],0,[00/1000 V 01/1000]


###Задача 2. Мили

In [9]:
#x0_2 = ['0', '0']
#2 входа - v
#2 состояния - x
#4 выхода - y

In [10]:
#Выходы и состояния для автомата Мили заданы функциями с логическими операторами

def x1(x, v):
  #v1 -> (v2 & (x2 v v1 v v1 ^ x2 + (v2 -> x1 ~ v2)))

  rezult = implic(int(v[0]), int(v[1]) and (pier(int(x[1]) or int(v[0]) or int(v[0]), int(x[1])) ^ equiv(implic(int(v[1]), int(x[0])), int(v[1]))))
  return rezult

def x2(x, v):
  #x1 | (x2 ^ (v1 ' | (v1 ~ (x1 + (x1 v v2)))))

  rezult = schaeffer(int(x[0]), pier(int(x[1]), schaeffer(not int(v[0]), equiv(int(v[0]), int(x[0]) ^ (int(x[0]) or int(v[1]))))))
  return rezult

def y1(x, v):
  #x1 + v1 -> (x2 ^ v1 | (x1'))

  rezult = implic(int(x[0]) ^ int(v[0]), schaeffer(pier(int(x[1]), int(v[0])), not int(x[0])))
  return rezult

def y2(x, v):
  #x2 v v2' -> (x2~v1)

  rezult = implic(int(x[1]) or (not int(v[1])), equiv(int(x[1]), int(v[0])))
  return rezult

def y3(x, v):
  #x1 & (x1 v (x2 & (x2 ^ (x1 & x2 + x1 | v2 & x1))))

  rezult = int(x[0]) and (int(x[0]) or (int(x[1]) and pier(int(x[1]), (int(x[0]) and int(x[1])) ^ (schaeffer(int(x[0]), int(v[1])) and int(x[0])))))
  return rezult

def y4(x, v):
  #v1 ^ x1' v (x1 v x2 -> x2 & (x2'))

  rezult = pier(int(v[0]), (not int(x[0])) or implic(int(x[0]) or int(x[1]), int(x[1]) and (not int(x[1]))))
  return rezult

In [11]:
print("Автомат Мили:\n")
truth_table2 = truthTable(pos_inputs, pos_states, x, v)
print("\n")
output_table2 = outputTable_Mili(pos_inputs, pos_states, x, v)

Автомат Мили:

Таблица истинности
+-------+----+----+----+----+
| v \ x | 00 | 01 | 10 | 11 |
+-------+----+----+----+----+
|   00  | 11 | 11 | 01 | 01 |
|   01  | 11 | 11 | 01 | 01 |
|   10  | 10 | 10 | 01 | 11 |
|   11  | 11 | 11 | 01 | 11 |
+-------+----+----+----+----+


Таблица выходов
+-------+------+------+------+------+
| v \ x |  00  |  01  |  10  |  11  |
+-------+------+------+------+------+
|   00  | 1100 | 1100 | 1000 | 1100 |
|   01  | 1000 | 1000 | 1100 | 1100 |
|   10  | 1111 | 1111 | 1010 | 1110 |
|   11  | 1011 | 1011 | 1110 | 1110 |
+-------+------+------+------+------+


In [12]:
truth_copy2 = pd.DataFrame(truth_table2.copy(), columns = pos_states)
truth_copy2['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=truth_copy2.index)

out_copy2 = pd.DataFrame(output_table2.copy(), columns = pos_states)
out_copy2['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=out_copy2.index)

creatly_connection_table(truth_copy2, out_copy2)

,00,01,10,11
0,0,0,0,0
1,0,0,00/1000 V 01/1100 V 10/1010 V 11/1110,00/1100 V 01/1100
2,10/1111,10/1111,0,0
3,00/1100 V 01/1000 V 11/1011,00/1100 V 01/1000 V 11/1011,0,10/1110 V 11/1110


##СКНФ/СДНФ


In [13]:
x = ['0', '1']
v = ['0', '1']


pos_inputs = []
for i in range(len(v)):
  for j in range(len(v)):
    pos_inputs.append(str(v[i]) + str(v[j]))
print('Возможные входы:', pos_inputs)

pos_states = []
for i in range(len(x)):
  for j in range(len(x)):
    pos_states.append(str(x[i]) + str(x[j]))
print('Возможные состояния:', pos_states)

Возможные входы: ['00', '01', '10', '11']
Возможные состояния: ['00', '01', '10', '11']


In [14]:
#СДНФ() (смотреть на 1)
def pdnf(inputTable, index, pos_inputs, pos_states, nameAvtomaton, tableType = 0):
  countStates = len(pos_states[0])
  countInputs = len(pos_inputs[0])
  countPosSt = len(pos_states)
  countPosInp = len(pos_inputs)
  rezultTable = pd.DataFrame()

  if(nameAvtomaton == "Мура" and tableType):
    countInputs = 1
    countPosInp = 1
    rezultTable = pd.DataFrame(it.product([0,1], repeat=(countStates)))
  else:
    rezultTable = pd.DataFrame(it.product([0,1], repeat=(countInputs + countStates)))

  countRows = len(rezultTable)
  function = "СДНФ = "

  rezultTable['f'] = np.zeros(countRows)
  rezultTable = np.array(rezultTable)

  k = 0
  logic = 0
  for i in range(countPosInp):
    for j in range(countPosSt):
      k = findIndex(rezultTable, pos_states[j], pos_inputs[i], nameAvtomaton, tableType)

      rezultTable[k][len(rezultTable[0]) - 1] = inputTable[i][j][index]

      if(int(inputTable[i][j][index])):
        if(logic):
          function += " V "
        function += formFunctionBlock(rezultTable[k], countInputs, countStates, " & ", nameAvtomaton, tableType) + "\n"
        logic = 1

  print(function)
  return rezultTable

#СКНФ() (смотреть на 0)
def pcnf(inputTable, index, pos_inputs, pos_states, nameAvtomaton, tableType = 0):
  countStates = len(pos_states[0])
  countInputs = len(pos_inputs[0])
  countPosSt = len(pos_states)
  countPosInp = len(pos_inputs)
  rezultTable = pd.DataFrame()

  if(nameAvtomaton == "Мура" and tableType):
    countInputs = 1
    countPosInp = 1
    rezultTable = pd.DataFrame(it.product([0,1], repeat=(countStates)))
  else:
    rezultTable = pd.DataFrame(it.product([0,1], repeat=(countInputs + countStates)))

  countRows = len(rezultTable)
  function = "СКНФ = "

  rezultTable['f'] = np.zeros(countRows)
  rezultTable = np.array(rezultTable)

  k = 0
  logic = 0
  for i in range(countPosInp):
    for j in range(countPosSt):
      k = findIndex(rezultTable, pos_states[j], pos_inputs[i], nameAvtomaton, tableType)

      rezultTable[k][len(rezultTable[0]) - 1] = inputTable[i][j][index]
      if(not int(inputTable[i][j][index])):
        if(logic):
          function += " & "
        function += formFunctionBlock(rezultTable[k], countInputs, countStates, " V ", nameAvtomaton, tableType) + "\n"
        logic = 1

  print(function)
  return rezultTable

def findIndex(table, x, v, nameAvtomaton, tableType = 0):
  vars = []
  table = np.array(table)

  for i in range(len(x)):
    vars += x[i]

  if(nameAvtomaton == "Мили" or not tableType):
    for i in range(len(v)):
      vars += v[i]

  logic = 0
  index = 0

  for i in range(len(table)):
    for j in range(len(vars)):
      if(int(table[i][j]) == int(vars[j])):
        logic = 1
      else:
        logic = 0
        break
    if(logic):
      index = i
      break

  return index

def formFunctionBlock(rowTable, countV, countX, sign, nameAvtomaton, tableType = 0):
  rezult = "("
  logic = "x"

  for i in range(countX):
    if(not rowTable[i]):
      logic += "'"

    if(i != countX - 1):
      rezult += logic + str(i + 1) + sign
    else:
      rezult += logic + str(i + 1)

    logic = "x"

  if(nameAvtomaton == "Мили" or not tableType):
    rezult += sign
    logic = "v"

    for i in range(countX):
      if(not rowTable[countV + i]):
        logic += "'"

      if(i != countV - 1):
        rezult += logic + str(i + 1) + sign
      else:
        rezult += logic + str(i + 1)

      logic = "v"

  rezult += ")"
  return rezult

###Задача 3. Автомат Мили

In [15]:
x0_3 = [1, 1]
phi = [['11', '01', '10', '10'], ['10', '00', '10', '10'], ['11', '01', '11', '11'], ['11', '01', '10', '10']]
psi = [['1101', '1110', '1100', '1110'], ['1010', '0111', '1111', '0111'], ['1001', '1110', '1101', '1111'], ['1000', '1101', '1101', '1101']]

#2 входа; 2 состояния; 4 выхода

In [16]:
print("Матрица переходов для автомата Мили:")
print(np.matrix(phi))
print("\n\nМатрица выходов для автомата Мили:")
print(np.matrix(psi))

Матрица переходов для автомата Мили:
[['11' '01' '10' '10']
 ['10' '00' '10' '10']
 ['11' '01' '11' '11']
 ['11' '01' '10' '10']]


Матрица выходов для автомата Мили:
[['1101' '1110' '1100' '1110']
 ['1010' '0111' '1111' '0111']
 ['1001' '1110' '1101' '1111']
 ['1000' '1101' '1101' '1101']]


In [17]:
truth_copy3 = pd.DataFrame(phi.copy(), columns = pos_states)
truth_copy3['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=truth_copy3.index)

out_copy3 = pd.DataFrame(psi.copy(), columns = pos_states)
out_copy3['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=out_copy3.index)

creatly_connection_table(truth_copy3, out_copy3)

,00,01,10,11
0,0,01/0111,0,0
1,0,00/1110 V 10/1110 V 11/1101,0,0
2,01/1010,0,00/1100 V 01/1111 V 11/1101,00/1110 V 01/0111 V 11/1101
3,00/1101 V 10/1001 V 11/1000,0,10/1101,10/1111


In [18]:
print("Для таблицы переходов при x_1:")
tablePDNF1 = pdnf(phi, 0, pos_inputs, pos_states, "Мили")
print()
tablePСNF1 = pcnf(phi, 0, pos_inputs, pos_states, "Мили")

print("\nДля таблицы переходов при x_2:")
tablePDNF2 = pdnf(phi, 1, pos_inputs, pos_states, "Мили")
tablePСNF2 = pcnf(phi, 1, pos_inputs, pos_states, "Мили")

print("\nДля таблицы выходов при y_1:")
tablePDNF3 = pdnf(psi, 0, pos_inputs, pos_states, "Мили", 1)
print()
tablePСNF3 = pcnf(psi, 0, pos_inputs, pos_states, "Мили", 1)

print("\nДля таблицы выходов при y_2:")
tablePDNF4 = pdnf(psi, 1, pos_inputs, pos_states, "Мили", 1)
print()
tablePСNF4 = pcnf(psi, 1, pos_inputs, pos_states, "Мили", 1)

print("\nДля таблицы выходов при y_3:")
tablePDNF5 = pdnf(psi, 2, pos_inputs, pos_states, "Мили", 1)
print()
tablePСNF5 = pcnf(psi, 2, pos_inputs, pos_states, "Мили", 1)

print("\nДля таблицы выходов при y_4:")
tablePDNF6 = pdnf(psi, 3, pos_inputs, pos_states, "Мили", 1)
print()
tablePСNF6 = pcnf(psi, 3, pos_inputs, pos_states, "Мили", 1)

Для таблицы переходов при x_1:
СДНФ = (x'1 & x'2 & v'1 & v'2)
 V (x1 & x'2 & v'1 & v'2)
 V (x1 & x2 & v'1 & v'2)
 V (x'1 & x'2 & v'1 & v2)
 V (x1 & x'2 & v'1 & v2)
 V (x1 & x2 & v'1 & v2)
 V (x'1 & x'2 & v1 & v'2)
 V (x1 & x'2 & v1 & v'2)
 V (x1 & x2 & v1 & v'2)
 V (x'1 & x'2 & v1 & v2)
 V (x1 & x'2 & v1 & v2)
 V (x1 & x2 & v1 & v2)


СКНФ = (x'1 V x2 V v'1 V v'2)
 & (x'1 V x2 V v'1 V v2)
 & (x'1 V x2 V v1 V v'2)
 & (x'1 V x2 V v1 V v2)


Для таблицы переходов при x_2:
СДНФ = (x'1 & x'2 & v'1 & v'2)
 V (x'1 & x2 & v'1 & v'2)
 V (x'1 & x'2 & v1 & v'2)
 V (x'1 & x2 & v1 & v'2)
 V (x1 & x'2 & v1 & v'2)
 V (x1 & x2 & v1 & v'2)
 V (x'1 & x'2 & v1 & v2)
 V (x'1 & x2 & v1 & v2)

СКНФ = (x1 V x'2 V v'1 V v'2)
 & (x1 V x2 V v'1 V v'2)
 & (x'1 V x'2 V v'1 V v2)
 & (x'1 V x2 V v'1 V v2)
 & (x1 V x'2 V v'1 V v2)
 & (x1 V x2 V v'1 V v2)
 & (x1 V x'2 V v1 V v2)
 & (x1 V x2 V v1 V v2)


Для таблицы выходов при y_1:
СДНФ = (x'1 & x'2 & v'1 & v'2)
 V (x'1 & x2 & v'1 & v'2)
 V (x1 & x'2 & v'1 & v'2)
 V 

###Задача 4. Автомат Мура

In [19]:
#x0_4 = ['0', '1']
phi_2 = [['00', '00', '10', '10'], ['10', '10', '10', '10'], ['01', '01', '11', '11'], ['00', '00', '10', '10']]
psi_2 = [['011', '010', '101', '011']]

#2 входа; 2 состояния; 3 выхода

In [20]:
print("Матрица переходов для автомата Мура:")
print(np.matrix(phi_2))
print("\n\nМатрица выходов для автомата Мура:")
print(np.matrix(psi_2))

Матрица переходов для автомата Мура:
[['00' '00' '10' '10']
 ['10' '10' '10' '10']
 ['01' '01' '11' '11']
 ['00' '00' '10' '10']]


Матрица выходов для автомата Мура:
[['011' '010' '101' '011']]


In [21]:
truth_copy4 = pd.DataFrame(phi_2.copy(), columns = pos_states, index = pos_inputs)
truth_copy4['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=truth_copy4.index)

out_copy4 = pd.DataFrame(np.matrix(psi_2).copy(), columns = pos_states)

creatly_connection_table(truth_copy4, out_copy4)

,00,01,10,11
00,[00/011 V 11/011],[00/010 V 11/010],0,0
01,[10/011],[10/010],0,0
10,[01/011],[01/010],[00/101 V 01/101 V 11/101],[00/011 V 01/011 V 11/011]
11,0,0,[10/101],[10/011]


In [22]:
print("Для таблицы переходов при x_1:")
tablePDNF7 = pdnf(phi_2, 0, pos_inputs, pos_states, "Мура")
print()
tablePСNF7 = pcnf(phi_2, 0, pos_inputs, pos_states, "Мура")

print("\nДля таблицы переходов при x_2:")
tablePDNF8 = pdnf(phi_2, 1, pos_inputs, pos_states, "Мура")
tablePСNF8 = pcnf(phi_2, 1, pos_inputs, pos_states, "Мура")

print("\nДля таблицы выходов при y_1:")
tablePDNF9 = pdnf(psi_2, 0, pos_inputs, pos_states, "Мура", 1)
print()
tablePСNF9 = pcnf(psi_2, 0, pos_inputs, pos_states, "Мура", 1)

print("\nДля таблицы выходов при y_2:")
tablePDNF10 = pdnf(psi_2, 1, pos_inputs, pos_states, "Мура", 1)
print()
tablePСNF10 = pcnf(psi_2, 1, pos_inputs, pos_states, "Мура", 1)

print("\nДля таблицы выходов при y_3:")
tablePDNF11 = pdnf(psi_2, 2, pos_inputs, pos_states, "Мура", 1)
print()
tablePСNF11 = pcnf(psi_2, 2, pos_inputs, pos_states, "Мура", 1)

Для таблицы переходов при x_1:
СДНФ = (x1 & x'2 & v'1 & v'2)
 V (x1 & x2 & v'1 & v'2)
 V (x'1 & x'2 & v'1 & v2)
 V (x'1 & x2 & v'1 & v2)
 V (x1 & x'2 & v'1 & v2)
 V (x1 & x2 & v'1 & v2)
 V (x1 & x'2 & v1 & v'2)
 V (x1 & x2 & v1 & v'2)
 V (x1 & x'2 & v1 & v2)
 V (x1 & x2 & v1 & v2)


СКНФ = (x'1 V x'2 V v'1 V v'2)
 & (x'1 V x2 V v'1 V v'2)
 & (x'1 V x'2 V v1 V v'2)
 & (x'1 V x2 V v1 V v'2)
 & (x'1 V x'2 V v1 V v2)
 & (x'1 V x2 V v1 V v2)


Для таблицы переходов при x_2:
СДНФ = (x'1 & x'2 & v1 & v'2)
 V (x'1 & x2 & v1 & v'2)
 V (x1 & x'2 & v1 & v'2)
 V (x1 & x2 & v1 & v'2)

СКНФ = (x'1 V x'2 V v'1 V v'2)
 & (x'1 V x2 V v'1 V v'2)
 & (x1 V x'2 V v'1 V v'2)
 & (x1 V x2 V v'1 V v'2)
 & (x'1 V x'2 V v'1 V v2)
 & (x'1 V x2 V v'1 V v2)
 & (x1 V x'2 V v'1 V v2)
 & (x1 V x2 V v'1 V v2)
 & (x'1 V x'2 V v1 V v2)
 & (x'1 V x2 V v1 V v2)
 & (x1 V x'2 V v1 V v2)
 & (x1 V x2 V v1 V v2)


Для таблицы выходов при y_1:
СДНФ = (x1 & x'2)


СКНФ = (x'1 V x'2)
 & (x'1 V x2)
 & (x1 V x2)


Для таблицы выходо